In [2]:
from datetime import date
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium import webdriver # driving the actions, links up with the browser
from bs4 import BeautifulSoup
import time
import re
from pick import pick
import progressbar
import os

In [8]:
# change the path

search_webpage = 'https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm'
#PATH = "D:\\Program Files (x86)\\chromedriver.exe"
PATH =  r'C:\Users\esthe\Documents\Coding\chromedriver_win32\chromedriver.exe'
#driver = webdriver.Chrome(PATH)
#driver.get(search_webpage)

In [4]:
yes = ["Yes", "Y", "YES", "y", "yes"]
starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
dashboard_url = "http://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
login_url = 'https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm'
output_name = 'WW_postings.csv'
choice = 'Application Deadlines in the next 10 Days'
potential_html1 = None
potential_html2 = None
potential_html3 = None

In [5]:
{"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 

{'Work Term Duration:': None,
 'Job Summary:': None,
 'Job Responsibilities:': None,
 'Required Skills:': None,
 'Targeted Degrees and Disciplines:': None,
 'Application Documents Required:': None}

In [6]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [7]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [25]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])

def main():
    starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"

    output_name = 'WW_postings.csv'
    
    browser = webdriver.Chrome(PATH)
    browser.get(starting_page) # in original script, this was starting page

    input('Are you ready?')

    
    data = get_job_lists(choice, browser, output_name)

    print("Done!")


def get_job_lists(choice, browser, output_name):
    
    print("Getting job lists ...")

    # Navigate to the job listings page then wait for the javascript to load
    browser.find_element_by_link_text(choice).click()

    input("Are you ready again?")
    
    # Get the HTML of the page and check if this is the last page (next_page_buttons)
    page_html = browser.execute_script("return document.body.innerHTML")
    pattern = r'<a href=".+?" onclick="loadPostingTable(.+?)">\s*»\s*<\/a>'
    next_page_buttons = []
    next_page_buttons = re.findall(pattern, page_html)



    # bar = progressbar.ProgressBar(max_value=self.job_lists_page_count)
    # bar.update(self.gather_current_progress)


    # dots for loading screen
    dots = ""
    counter_page = 0
    # Scrape the tables and save it to output_name as a CSV
    job_posting_importants_order = ['Work Term Duration:', "Job Summary:", "Job Responsibilities:", "Required Skills:", 
                                    "Targeted Degrees and Disciplines:", "Application Documents Required:"]
    with open(output_name, 'w') as f:
        row_count = 0
        job_count = 0
        while (counter_page < 1):
            counter_page += 1
            # loading screen stuff
            os.system('clear')
            print("Working" + dots)
            # update dots
            if (dots == "..."):
                dots = ""
            else:
                dots += "."

            # wait for JavaScript to load then download the page info
            time.sleep(2)
            soup = BeautifulSoup(page_html, "html.parser")
            
            for tr in soup.find_all('tr')[2:]: # iterating throw all rows
            
                
                job_count += 1
                print(job_count)
                try:
                    # all the data that was not in the original script
                    job_posting_importants = {"Work Term Duration:": None, # might need to do some regex filtering
                                             "Job Summary:" : None,
                                             "Job Responsibilities:": None,
                                             "Required Skills:" : None,
                                             "Targeted Degrees and Disciplines:": None,
                                             "Application Documents Required:": None} 


                    tds = tr.find_all('td') # iterating all cells in each row.
                    count = 0
                    job_info_list = []
                    for x in tds[0:-1]: # starting with the 3 buttons


                        if (count == 0): # at 3 buttons
                            link_container = x.find_all('a')[2]

                            onclick_link = link_container['onclick']

                            # clicking the onclick link:
                            browser.execute_script(onclick_link)
                            browser.switch_to.window(browser.window_handles[1])

                            # give the browser 1 seconds for the website to catch up with the code
                            browser.implicitly_wait(2)

                            time.sleep(3)
                            # do stuff, locate the texts of 'job posting importants'

                            #saving the contents of the html of the new page:
                            indiv_html = browser.execute_script("return document.body.innerHTML")
                            indiv_posting = BeautifulSoup(indiv_html, "html.parser")

                            # finding the third 'table' in the page, as the first table is no use to us

                            job_posting_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[1]


                            
                            #print(job_posting_information_table)


                            # iterating through all rows in the table
                            for row in job_posting_information_table.find_all('tr'): 
                                cell_list = row.find_all('td')
                                try:
                                    key = cell_list[0].get_text().strip()
                                except:
                                    continue


                                if (key in job_posting_importants):
                                    try:
                                        job_posting_importants[key] = cell_list[1].get_text().strip()
                                    except:
                                        job_posting_importants[key] = ""


                            application_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[2]
                            job_posting_importants["Application Documents Required:"] = application_information_table.find_all('td')[3].get_text().strip()

                            browser.close() # close current tab
                            browser.switch_to.window(browser.window_handles[0])

                            #print("lala")
                        if (count >= 2):  
                            #print(x.text)
                            job_info_list.append(str(x.text))

                        count += 1


                        
                    for col in job_posting_importants_order:
                        job_info_list.append(job_posting_importants[col])

                    df.loc[row_count] = job_info_list
                    row_count+=1
                except:
                    continue
                
            # navigate to the next page and get the HTML, next page buttons
            next_page_buttons = re.findall(pattern, page_html)
            browser.find_element_by_link_text("»").click()
            time.sleep(2)
            page_html = browser.execute_script("return document.body.innerHTML")


if __name__ == "__main__":
    main()

Are you ready?y
Getting job lists ...
Are you ready again?y
Working
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0)" onclick="loadPostingTable('', '', 'Forward', '7','postingsExpIn10','', null)">...</a> is not clickable at point (460, 10). Other element would receive the click: <div class="navbar-inner">...</div>
  (Session info: chrome=87.0.4280.141)


In [26]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185577,\n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Planner\n\...,City of St Catharines,Municipal Office,1,Open for Applications,St. Catharines,"Intermediate, Senior",12,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Student Planner,To research assigned projects for data relevan...,Students should be experienced in virtual meet...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,186774,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,City of Waterloo,Development Services,1,Open for Applications,Waterloo,"Intermediate, Senior",63,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,The successful candidate will have involvement...,Growth Management Projects may include:\n\nLan...,The candidate should have a strong working kno...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,185901,\n\n\n\n\t\t\t\t\t\t\t\t\t\tFleet Assistant\n\...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,9,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,A co-op opportunity exists for a student to su...,"Provide support to the Fleet Coordinator, Flee...",Currently enrolled in a relevant post-secondar...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
3,186759,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,17,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,A co-op position exists for an individual to a...,Assist with park bookings and event scheduling...,1-2 years of related post-secondary education ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
4,185951,\n\n\n\n\t\t\t\t\t\t\t\t\t\tCommunity Developm...,City of Windsor,Divisional Office,1,Open for Applications,Windsor,"Junior, Intermediate, Senior",18,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,We are looking for an eager and dedicated indi...,Under the supervision and direction of the Man...,The successful applicant will have an excellen...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,184464,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst, eDiscover...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",7,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Learn from deep subject matter experts through...,About the team \n \nWith the proliferation of ...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
96,184476,\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst Intern - I...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Partner with clients to solve their most compl...,About the team \nThe Innovation & Technology C...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
97,184485,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst, Financial...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",23,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Learn from deep subject matter experts through...,About the team \n \nThe Financial Engineering ...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"Un

In [27]:
df.to_csv('page6.csv')

In [68]:
#df = pd.read_csv('half_data.csv')
df2 = pd.read_csv('page6.csv')


In [73]:
df2


,Unnamed: 0,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185577,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of St Catharines,Municipal Office,1,Open for Applications,St. Catharines,"Intermediate, Senior",12,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,1,186774,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Development Services,1,Open for Applications,Waterloo,"Intermediate, Senior",63,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,2,185901,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,9,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
3,3,186759,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,17,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
4,4,185951,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Windsor,Divisional Office,1,Open for Applications,Windsor,"Junior, Intermediate, Senior",18,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,184464,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",7,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
96,96,184476,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",20,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
97,97,184485,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",23,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
98,98,184489,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",11,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op W

In [ ]:
a = {"bob": 2, 'sam':6, 'asd':123}
a.values()

In [ ]:
' '.join("j, i, s".split())

In [ ]:
e2jin@uwaterloo.ca
Eevee1218.

In [ ]:
'''
Things to Change
0. Get rid of whitespaces for job title - DONE
1. Get rid of capital NEW at the start of job titles - DONE
2. get rid of divison - DONE 
3. get rid of internal status - DONE
4. turn level into 3 binary columns - Dropped it, don't think its that important
5. filter all until only app deadline <= jan 19 - DONE
6. whitespace for targeted clusters - DONE
7. App docs required turns into need cover letter? - DONE
8. change Work Term Duration to [1,4]
9. whitepsace for app deadlinke - DONE

'''

In [70]:
def surrounding_space_remover(str):
    str = str.replace('\n','')
    str = str.replace('\t','')
    return str
    
df_space_remover = np.vectorize(surrounding_space_remover)

In [71]:
for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    print(column)
    df2[column] = df_space_remover(str(df2[column]))
    

Job Title
App Deadline
Targeted Degrees and Disciplines
Job Summary
Job Responsibilities
Required Skills
Targeted Degrees and Disciplines


In [72]:
df2['Targeted Degrees and Disciplines']

0     0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
1     0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
2     0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
3     0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
4     0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
                            ...                        
95    0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
96    0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
97    0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
98    0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
99    0     0     Targeted Clusters\n\t\t\t\n\n\t\t\...
Name: Targeted Degrees and Disciplines, Length: 100, dtype: object

In [52]:
def new_removal(str):
    try:
        if (str[0:3] == "NEW"): 
            return str[3:]
    except:
        return str # not enough chars in job title
    return str # if no NEW in job title

df_new_remover = np.vectorize(new_removal)

In [53]:
df['Job Title'] = df_new_remover(df["Job Title"])

In [54]:
def day_extractor(str):
    str = str.replace(",", " ")
    str = [int(s) for s in str.split() if s.isdigit()]
    return str[0]

df_day_extractor = np.vectorize(day_extractor)

In [55]:
df['App Deadline'] = df_day_extractor(df["App Deadline"])
df = df[df["App Deadline"] <= 19]

In [56]:
def cover_letter_needed(str):
    return ("Cover Letter" in str)

df_cover_letter_needed = np.vectorize(cover_letter_needed)

In [58]:
df['Application Documents Required'] = df_cover_letter_needed(str(df['Application Documents Required']))

In [59]:
df.rename(columns={"Application Documents Required": "Cover Letter Needed"}, inplace=True)

In [60]:
def work_term_duration(str):
    if (str == '4 month work term'): return 1
    if (str == '8 month consecutive work term required'): return 2
    if (str == '8 month consecutive work term preferred'): return 3
    else: return 4 # probably 4 month work term preferred?
    
df_work_term_duration = np.vectorize(work_term_duration)

In [61]:
df["Work Term Duration"] = df_work_term_duration(df["Work Term Duration"])

In [62]:
df

,Unnamed: 0,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,0.0,186082.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,BlackBerry,Head Office,1.0,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75.0,0,1,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
1,1.0,186083.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,BlackBerry,Head Office,1.0,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69.0,0,1,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
2,2.0,186087.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,BlackBerry,Head Office,1.0,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3.0,0,2,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
3,3.0,186091.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,BlackBerry,Head Office,1.0,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40.0,0,1,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
4,4.0,186095.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,BlackBerry,Head Office,1.0,Open for Applications,Waterloo,"Junior, Intermediate",23.0,0,1,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,133.0,188060.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,Zoocasa,Head Office,1.0,Open for Applications,Toronto,"Junior, Intermediate",12.0,0,1,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
1335,134.0,185591.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,Zymeworks Inc.,Divisional Office,1.0,Open for Applications,Vancouver,"Junior, Intermediate",7.0,0,2,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
1336,135.0,185600.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,Zymeworks Inc.,Divisional Office,1.0,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3.0,0,2,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
1337,136.0,185603.0,0 \n\n\n\t\t\t\t\t\t\t\t\t\tData Scienti...,Zymeworks Inc.,Divisional Office,1.0,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4.0,0,3,0 POSITION SUMMARY\n\nThis role is in Bl...,0 RESPONSIBILITIES\n\nDevelop novel ML c...,0 SKILLS AND QUALIFICATIONS\n\nCurrently...,0 0 Targeted Clusters\n\t\t\t\n\n\...,False
